In [131]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve
import numpy as np
import plotly.express as px
# import shap
import seaborn as sns
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    multilabel_confusion_matrix,
)
from catboost.utils import select_threshold
from skmultilearn.model_selection.iterative_stratification import (
    IterativeStratification,
)
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm import tqdm
from catboost.utils import get_roc_curve, select_threshold
from torchmetrics.classification import BinaryPrecisionRecallCurve
# from supervised.automl import AutoML
import warnings
from loguru import logger
from sklearn.metrics import f1_score, precision_score, recall_score
from catboost.utils import eval_metric
from catboost import metrics as cat_metrics
from sklearn.utils import class_weight

warnings.filterwarnings("ignore")


In [132]:
wr.config.s3_endpoint_url = "http://localhost:8333"

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df_3/",
    dataset=True,
)
multilabelDf.columns

Index(['vision_bonnet', 'vision_bumper_front', 'vision_engine',
       'vision_grille', 'vision_headlamp_lh', 'vision_headlamp_rh',
       'vision_door_front_lh', 'vision_door_front_rh', 'vision_bumper_rear',
       'vision_misc', 'vision_front_panel', 'vision_non_external',
       'vision_wheel', 'vision_fender_front_lh', 'vision_fender_front_rh',
       'vision_rear_quarter_lh', 'vision_tail_lamp_lh', 'vision_tail_lamp_rh',
       'vision_windscreen_front', 'vision_rear_compartment',
       'vision_rear_panel', 'vision_rear_quarter_rh', 'vision_door_rear_rh',
       'vision_door_rear_lh', 'vision_windscreen_rear', 'bonnet',
       'bumper_front', 'air_conditioning', 'cooling_fan', 'grille',
       'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh',
       'radiator', 'bumper_rear', 'misc', 'engine', 'airbag', 'front_panel',
       'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh',
       'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh',
       

In [133]:
# multilabelDf.replace(pd.NA, 0, inplace=True)

In [134]:
multilabelDf["Assembly_Type"]

0         3
1         1
2         3
3         3
4         3
         ..
583810    3
583811    3
583812    3
583813    3
583814    3
Name: Assembly_Type, Length: 583815, dtype: Int64

In [135]:
multilabelDf[multilabelDf["Model"] == "NA"]

,vision_bonnet,vision_bumper_front,vision_engine,vision_grille,vision_headlamp_lh,vision_headlamp_rh,vision_door_front_lh,vision_door_front_rh,vision_bumper_rear,vision_misc,...,Assembly_Type,Vehicle_Still_Driveable,NCB_Stat,Claim_Type,Vehicle_Type,Sum_Insured,Repairer,Repairer_Apprv_Count,Collision_With,Handling_Insurer


In [136]:
wr.config.s3_endpoint_url = "http://localhost:8333"

multilabelDf = wr.s3.read_parquet(
        path=f"s3://multilabel_df_3/",
        dataset=True,
    )
allVisionFeatures = [x for x in multilabelDf.columns if "vision_" in x]
caseFeatures = [
    "Circumstances_of_Accident",
    "Model",
    # "Vehicle_Still_Driveable",
    # "NCB_Stat",
    # "Assembly_Type",
    # "Claim_Type",
    "Vehicle_Type",
    # # "Sum_Insured",
    # "Repairer",
    # # # "Repairer_Apprv_Count",
    # "Collision_With",
    # "Handling_Insurer",
]
exclude = ['Sum_Insured', 'Repairer', 'Repairer_Apprv_Count', 
"Vehicle_Still_Driveable",
    "NCB_Stat",
    "Assembly_Type",
    "Claim_Type",
    # "Vehicle_Type",
    "Sum_Insured",
    "Repairer",
    "Repairer_Apprv_Count",
    "Collision_With",
    "Handling_Insurer"
     ]
# contFeatures = ["Sum_Insured", "Repairer_Apprv_Count"]
allInputFeature = caseFeatures + allVisionFeatures
allTargetCol = [
   'bonnet', 'bumper_front', 'grille', 'fog_lamp_rh', 'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh', 'air_conditioning', 'cooling_fan', 'radiator', 'bumper_rear', 'misc', 'engine', 'airbag', 'front_panel', 'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh', 'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'fog_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_mirror_lh', 'door_rear_lh', 'windscreen_rear'
]

# allTargetCol = [
#    'bonnet', 
#    'bumper_front', 
#    'grille', 
#    'headlamp_lh',
#    'headlamp_rh', 
#    'door_front_lh', 
#    'door_front_rh', 
#    'bumper_rear', 
#    'engine', 
#    'front_panel', 
#    'wheel',
#    'fender_front_lh',
#    'fender_front_rh', 
#    'rear_quarter_lh', 
#    'tail_lamp_lh', 
#    'tail_lamp_rh',
#    'windscreen_front', 
#    'rear_compartment',
#    'rear_panel', 
#    'rear_quarter_rh', 
#    'door_rear_rh', 
#    'door_rear_lh', 
#    'windscreen_rear'
# ]
saloonPredDf = pd.read_csv(
    "/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/116/Saloon - 4 Dr/Saloon - 4 Dr_imgs_pred_output.csv"
)
hatchBackDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/116/Hatchback - 5 Dr/Hatchback - 5 Dr_imgs_pred_output.csv")
suvDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/116/SUV - 5 Dr/SUV - 5 Dr_imgs_pred_output.csv")

imgPredOutput = pd.concat([saloonPredDf, hatchBackDf, suvDf])
# imgPredOutput = saloonPredDf
allPredModel = []
allPreds = []
allGt = []
realTestDataDf = multilabelDf[caseFeatures + allTargetCol + ["CaseID"]].merge(
    imgPredOutput, on="CaseID"
)
realTestDataDf = realTestDataDf.loc[
    :, ~realTestDataDf.columns.str.contains("^Unnamed")
]
trainDf = multilabelDf[
    ~multilabelDf["CaseID"].isin(realTestDataDf["CaseID"].unique().tolist())
]
# trainDf = trainDf.sample(frac=0.5)
assert set(realTestDataDf["CaseID"].tolist()).isdisjoint(trainDf["CaseID"].tolist())
targetPart = [
    "fog_lamp_rh",
    "airbag",
    "undercarriage",
    "door_mirror_rh",
    "interior",
    "fog_lamp_lh",
    "door_mirror_lh",
    "windscreen_rear",
    "air_conditioning",
    "cooling_fan",
    "radiator"
]
# targetPart = [
#      'bonnet', 'bumper_front', 'grille', 'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh', 'bumper_rear', 'misc', 'engine', 'front_panel', 'wheel', 'fender_front_lh', 'fender_front_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_rear_lh'
# ]
# targetCol = list(set(targetPart) & set(allTargetCol))
targetCol = allTargetCol


In [137]:
allVisionFeatures

['vision_bonnet',
 'vision_bumper_front',
 'vision_engine',
 'vision_grille',
 'vision_headlamp_lh',
 'vision_headlamp_rh',
 'vision_door_front_lh',
 'vision_door_front_rh',
 'vision_bumper_rear',
 'vision_misc',
 'vision_front_panel',
 'vision_non_external',
 'vision_wheel',
 'vision_fender_front_lh',
 'vision_fender_front_rh',
 'vision_rear_quarter_lh',
 'vision_tail_lamp_lh',
 'vision_tail_lamp_rh',
 'vision_windscreen_front',
 'vision_rear_compartment',
 'vision_rear_panel',
 'vision_rear_quarter_rh',
 'vision_door_rear_rh',
 'vision_door_rear_lh',
 'vision_windscreen_rear']

In [138]:

for part in tqdm(targetCol):
    logger.success(f"Start training for part : {part}")
    allCaseIdByPart = []
    allPredByPart = []
    allGtByPart = []
    trainCaseId = trainDf["CaseID"].tolist()
    tempTestDataDf = realTestDataDf
    # tempTestDataDf = realTestDataDf.groupby(part).head(5000)
    testCaseId = tempTestDataDf["CaseID"].tolist()
    # sampledDf = trainDf[allInputFeature + [part]].groupby(part).sample(frac=0.3)
    X_train = trainDf[allInputFeature]
    Y_train = trainDf[part].to_frame()
   
    X_test = tempTestDataDf[allInputFeature]
    Y_test = tempTestDataDf[part].to_frame()

    pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
    neg_count = len(Y_test[Y_test[part] == 0]) / len(Y_test)
    pos_weight = neg_count / pos_count
    train_pool = Pool(
        X_train, Y_train, cat_features=caseFeatures + allVisionFeatures
    )
    test_pool = Pool(X_test, Y_test, cat_features=caseFeatures + allVisionFeatures)
    clsWeight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(Y_train.values),y=Y_train.values.squeeze(1))
    logger.success(f"train_class_weight : {clsWeight}")

    testClsWeights = {
        0 : clsWeight[0],
        1 : clsWeight[1]

    }
    print(testClsWeights)
    sampleWeights = class_weight.compute_sample_weight(class_weight=testClsWeights, y=Y_test.values)
    print(f"Shape :{sampleWeights.shape}")
    # prCurve = MulticlassPrecisionRecallCurve(num_classes=2, thresholds=11)
    clf = CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="F1",
            iterations=300,
            early_stopping_rounds=100,
            task_type="GPU",
            class_weights= clsWeight,
            # auto_class_weights="SqrtBalanced",
            use_best_model=True,
            # scale_pos_weight=20
            # class_names=["not_dmg", "dmg"],
        )
    clf.fit(train_pool, eval_set=test_pool, metric_period=50, plot=False, verbose=1)
    test_predict = clf.predict(test_pool)
    # if (part in partFPThreshold):
    #     logger.success(f"Trade off false positive : {part}")
    #     (fpr, tpr, thresholds) = get_roc_curve(clf, test_pool, plot=False)
    #     boundary = select_threshold(clf, curve=(fpr, tpr, thresholds), FPR=partFPThreshold[part])
    #     clf.set_probability_threshold(boundary)
    #     test_predict = clf.predict(X_test)
    # elif part in partFNThreshold:
    #     logger.success(f"Trade off false negative : {part}")
    #     (fpr, tpr, thresholds) = get_roc_curve(clf, test_pool, plot=False)
    #     boundary = select_threshold(clf, curve=(fpr, tpr, thresholds), FNR=partFNThreshold[part])
    #     clf.set_probability_threshold(boundary)
    #     test_predict = clf.predict(X_test)
    acc = accuracy_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    confMat = confusion_matrix(Y_test.values.astype(np.int64), test_predict,  sample_weight=sampleWeights)
    pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
    pos_count_abs = len(Y_test[Y_test[part] == 1])
    # catboostF1 = cat_metrics.F1(Y_test.values.astype(np.int64), test_predict)
    tn = confMat[0][0]
    tp = confMat[1][1]
    fp = confMat[0][1]
    fn = confMat[1][0]
    totalSample = fp + fn + tp + tn
    acc = (tp + tn) / (fp + fn + tp + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 =  (2 * precision * recall) / (precision + recall)
    f1_weighted = f1_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    # precision_weighted = precision_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    # recall_weighted = recall_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)

    logger.success(f"F1 for part : {part} = {f1}")
    logger.success(f"Weighted F1 scikit for part : {part} = {f1_weighted}")

    # assert (tp / (tp + fn)) + (fn / (tp + fn)) == (tn / (tn + fp)) + (
    #     fp / (tn + fp)
    # )

    allPredModel.append(
        {
            "part": part,
            "tp": tp / (tp + fn),
            "tn": tn / (tn + fp),
            "fp": fp / (tn + fp),
            "fn": fn / (tp + fn),
            "acc": acc,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "pos_count": pos_count,
            "pos_count_abs": pos_count_abs,

        }
    )
    assert len(testCaseId) == len(test_predict)
    assert len(testCaseId) == len(Y_test)

    allPreds.append({"CaseID": testCaseId, part: test_predict})
    allGt.append({"CaseID": testCaseId, part: Y_test.values.squeeze(1)})


  0%|          | 0/34 [00:00<?, ?it/s]2023-01-03 11:21:50.774 | SUCCESS  | __main__:<module>:2 - Start training for part : bonnet
2023-01-03 11:21:52.114 | SUCCESS  | __main__:<module>:25 - train_class_weight : [0.78312139 1.38301346]


{0: 0.7831213911233399, 1: 1.383013462911}
Shape :(17991,)
Learning rate set to 0.068394
0:	learn: 1.0000000	test: 0.8537668	best: 0.8537668 (0)	total: 19.5ms	remaining: 5.82s
50:	learn: 1.0000000	test: 0.8537668	best: 0.8537668 (0)	total: 1.29s	remaining: 6.32s


2023-01-03 11:21:54.984 | SUCCESS  | __main__:<module>:80 - F1 for part : bonnet = 0.8537667540726704
2023-01-03 11:21:54.984 | SUCCESS  | __main__:<module>:81 - Weighted F1 scikit for part : bonnet = 0.8537667540727084
  3%|▎         | 1/34 [00:04<02:18,  4.21s/it]2023-01-03 11:21:54.985 | SUCCESS  | __main__:<module>:2 - Start training for part : bumper_front


100:	learn: 1.0000000	test: 0.8537668	best: 0.8537668 (0)	total: 2.54s	remaining: 5s
bestTest = 0.853766752
bestIteration = 0
Shrink model to first 1 iterations.


2023-01-03 11:21:56.295 | SUCCESS  | __main__:<module>:25 - train_class_weight : [1.39822178 0.7783277 ]


...

100:	learn: 1.0000000	test: 0.3308156	best: 0.3308156 (0)	total: 2.2s	remaining: 4.33s
bestTest = 0.3308156299
bestIteration = 0
Shrink model to first 1 iterations.


2023-01-03 11:24:39.130 | SUCCESS  | __main__:<module>:25 - train_class_weight : [ 0.5212825  12.24674257]


{0: 0.5212824958588451, 1: 12.246742565256914}
Shape :(17991,)
Learning rate set to 0.068394
0:	learn: 1.0000000	test: 0.7079209	best: 0.7079209 (0)	total: 18.8ms	remaining: 5.62s
50:	learn: 1.0000000	test: 0.7079209	best: 0.7079209 (0)	total: 1.09s	remaining: 5.32s


2023-01-03 11:24:41.559 | SUCCESS  | __main__:<module>:80 - F1 for part : windscreen_rear = 0.7079208614702223
2023-01-03 11:24:41.559 | SUCCESS  | __main__:<module>:81 - Weighted F1 scikit for part : windscreen_rear = 0.7079208614702185
100%|██████████| 34/34 [02:50<00:00,  5.02s/it]

100:	learn: 1.0000000	test: 0.7079209	best: 0.7079209 (0)	total: 2.11s	remaining: 4.15s
bestTest = 0.7079208614
bestIteration = 0
Shrink model to first 1 iterations.


In [139]:

evalMetrics = pd.json_normalize(allPredModel)
avgPrecision = evalMetrics["precision"].mean()
avgRecall = evalMetrics["recall"].mean()
avgF1 = evalMetrics["f1"].mean()
avgTp = evalMetrics["tp"].mean()
avgFp = evalMetrics["fp"].mean()

avgTn = evalMetrics["tn"].mean()
avgAcc = evalMetrics["acc"].mean()
avgFn = evalMetrics["fn"].mean()
avgPosCount = evalMetrics["pos_count"].mean()

print(f"Avg Precision : {avgPrecision}")
print(f"Avg Recall : {avgRecall}")
print(f"Avg F1 : {avgF1}")
print(f"Avg TP : {avgTp}")
print(f"Avg TN : {avgTn}")
print(f"Avg FN : {avgFn}")
print(f"Avg pos_count : {avgPosCount}")

print(f"avgAcc : {avgAcc}")

Avg Precision : 0.8519171523612237
Avg Recall : 0.6005470635577501
Avg F1 : 0.6741492176290521
Avg TP : 0.6005470635577501
Avg TN : 0.9028454234030576
Avg FN : 0.3994529364422497
Avg pos_count : 0.15311086916007022
avgAcc : 0.7986651979047371


In [140]:
fig = px.bar(evalMetrics, x = 'part', y = 'pos_count_abs', text_auto=True)
fig

# fig.update_layout(
#     yaxis = dict(
#         tickmode = 'array',
#         tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
#     )
# )

In [141]:
fig = px.bar(evalMetrics, x = 'part', y = 'pos_count')

fig.add_hline(y=avgPosCount, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)

In [142]:
fig = px.bar(evalMetrics, x = 'part', y = 'tp')

fig.add_hline(y=avgTp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [143]:
fig = px.bar(evalMetrics, x = 'part', y = 'fp')

fig.add_hline(y=avgFp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgFp, 2), 0.2, 0.5, 0.7],
    )
)

In [144]:
fig = px.bar(evalMetrics, x = 'part', y = 'tn')

fig.add_hline(y=avgTn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [145]:
fig = px.bar(evalMetrics, x = 'part', y = 'fn')

fig.add_hline(y=avgFn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgFn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [146]:
fig = px.bar(evalMetrics, x = 'part', y = 'acc')

fig.add_hline(y=avgAcc, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgAcc, 2), 0.2, 0.5, 0.7],
    )
)

In [147]:
fig = px.bar(evalMetrics, x = 'part', y = 'precision')

fig.add_hline(y=avgPrecision, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgPrecision, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [148]:
fig = px.bar(evalMetrics, x = 'part', y = 'recall')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [149]:
fig = px.bar(evalMetrics, x = 'part', y = 'f1')

fig.add_hline(y=avgF1, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgF1, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [150]:
multilabelPred = pd.DataFrame(pd.DataFrame(allPreds[0]))
for p in tqdm(allPreds[1:]):
    dfPart = pd.DataFrame(p)
    multilabelPred = multilabelPred.merge(dfPart, on="CaseID")
multilabelPred.sort_values(by="CaseID", inplace=True)
multilabelPred

100%|██████████| 33/33 [00:00<00:00, 33.66it/s]


,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
0,10000038,1,1,1,1,1,1,0,0,1,...,1,1,0,1,1,0,0,1,0,0
1,10000084,0,0,0,0,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,0
2,10000104,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000117,0,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,10000181,1,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17986,13561462,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17987,13561785,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17988,13562091,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
17989,13562279,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
multilabelGt = pd.DataFrame(pd.DataFrame(allGt[0]))
for p in tqdm(allGt[1:]):
    dfPart = pd.DataFrame(p)
    multilabelGt = multilabelGt.merge(dfPart, on="CaseID")
multilabelGt.sort_values(by="CaseID", inplace=True)

100%|██████████| 33/33 [00:01<00:00, 31.54it/s]


In [152]:
multilabelGt = multilabelGt[multilabelPred.columns]
multilabelGt

,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
0,10000038,1,1,1,0,0,0,0,0,1,...,0,1,0,1,1,0,0,0,0,0
1,10000084,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
2,10000104,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10000117,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10000181,1,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17986,13561462,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17987,13561785,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
17988,13562091,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
17989,13562279,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = np.divide(len(set_true.intersection(set_pred)), 
                    float( len(set_true) ))
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean( [x for x in acc_list if not np.isnan(x)]), acc_list

In [154]:
# tempTargetCol = ["airbag", "undercarriage", "interior"]

In [155]:
subsetAcc2, acc_list = hamming_score(multilabelGt[targetCol].values, multilabelPred[targetCol].values)
subsetAcc2

0.6145170344097947

In [156]:
evalData = pd.DataFrame(
    [
        {
            "metric" : "subset_accuracy",
            "value" : subsetAcc2

        },
         {
            "metric" : "avg_f1",
            "value" : avgF1

        },
         {
            "metric" : "avg_tpr",
            "value" : avgTp

        },
        {
            "metric" : "avg_tnr",
            "value" : avgTn

        },
        {
            "metric" : "part_recall",
            "value" : avgRecall

        },
            {
            "metric" : "part_precision",
            "value" : avgPrecision

        },
                {
            "metric" : "part_accuracy",
            "value" : avgAcc

        }
    ]
)


In [157]:
fig = px.bar(evalData, x = 'metric', y = 'value', text_auto=True)
fig

In [158]:
corr = evalMetrics["tp"].corr(evalMetrics["pos_count"])
corr

0.6351953922588855

In [159]:
corr = evalMetrics["f1"].corr(evalMetrics["pos_count"])
corr

0.5157488613658348

In [160]:
corr = evalMetrics["acc"].corr(evalMetrics["pos_count"])
corr

-0.06922193614476438

In [161]:
targetCol[-1]

'windscreen_rear'

In [162]:
clf.classes_

array([0, 1])

In [163]:
# explainer = shap.Explainer(clf)
# shap_values = explainer(X_test)
# plt.title(targetCol[-1])
# shap.plots.beeswarm(shap_values)


In [164]:
currCaseStudyDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_2/case_study_gt.csv")
currCaseStudyDf

,Unnamed: 0,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
0,5025,10053736,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7519,10142635,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19780,10162786,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,1,0
3,3060,10228504,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,23273,10229726,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,11606,10231429,0,1,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
6,4379,10284983,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,12628,10293939,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,12685,10301735,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,13748,10353034,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
caseStudyPred = multilabelPred[multilabelPred['CaseID'].isin(currCaseStudyDf["CaseID"].unique().tolist())]
caseStudyGt = multilabelGt[multilabelGt["CaseID"].isin(currCaseStudyDf["CaseID"].unique().tolist())]
caseStudyPred.to_csv("case_study_pred.csv")
caseStudyGt.to_csv("case_study_gt.csv")

In [166]:
caseStudyPred

,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
711,10053736,0,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2040,10142635,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2370,10162786,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
3319,10228504,1,1,0,1,1,1,0,0,1,...,1,0,0,0,0,0,0,1,0,0
3346,10229726,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3387,10231429,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4309,10284983,1,1,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4468,10293939,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4584,10301735,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5451,10353034,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
